# Data Preprocessing

**Prerequisite : EDA.ipynb**

In [214]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

# Algorithms
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score

In [162]:
df = pd.read_excel("../artifacts/raw/E Commerce Dataset.xlsx", sheet_name="E Comm", index_col=0)

### One Hot Encoding

In [163]:
# One hot encoding to prepare for iterative imputation
categorical_columns = ["PreferredLoginDevice", "CityTier", "PreferredPaymentMode", "Gender", "PreferedOrderCat", "MaritalStatus"] # for one hot encoding, exclude columns that are already binary
numerical_columns = [col for col in df.columns if col not in categorical_columns] # Churnhas been dropped above
df_one_hot = pd.get_dummies(df, columns=categorical_columns, dtype=int)

df_one_hot.head()

,Churn,Tenure,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,...,Gender_Male,PreferedOrderCat_Fashion,PreferedOrderCat_Grocery,PreferedOrderCat_Laptop & Accessory,PreferedOrderCat_Mobile,PreferedOrderCat_Mobile Phone,PreferedOrderCat_Others,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
CustomerID,,,,,,,,,,,,,,,,,,,,,
50001,1,4.0,6.0,3.0,3,2,9,1,11.0,1.0,...,0,0,0,1,0,0,0,0,0,1
50002,1,NaN,8.0,3.0,4,3,7,1,15.0,0.0,...,1,0,0,0,1,0,0,0,0,1
50003,1,NaN,30.0,2.0,4,3,6,1,14.0,0.0,...,1,0,0,0,1,0,0,0,0,1
50004,1,0.0,15.0,2.0,4,5,8,0,23.0,0.0,...,1,0,0,1,0,0,0,0,0,1
50005,1,0.0,12.0,NaN,3,5,3,0,11.0,1.0,...,1,0,0,0,1,0,0,0,0,1


### Handle Null values

In [164]:
# Copy of dataset with dropped null values
df_dropped = df_one_hot.dropna()

print("Before: ", df_one_hot.shape)
print("After: ", df_dropped.shape)
print("Dropped: ", df_one_hot.shape[0] - df_dropped.shape[0])

Before:  (5630, 37)
After:  (3774, 37)
Dropped:  1856


In [165]:
# Imputing with Iterative Imputer
imputer = IterativeImputer(random_state=0)
df_imputed = pd.DataFrame(imputer.fit_transform(df_one_hot), columns=df_one_hot.columns)

df_imputed.head()

,Churn,Tenure,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,...,Gender_Male,PreferedOrderCat_Fashion,PreferedOrderCat_Grocery,PreferedOrderCat_Laptop & Accessory,PreferedOrderCat_Mobile,PreferedOrderCat_Mobile Phone,PreferedOrderCat_Others,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,1.0,4.000000,6.0,3.000000,3.0,2.0,9.0,1.0,11.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,2.451829,8.0,3.000000,4.0,3.0,7.0,1.0,15.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,3.649009,30.0,2.000000,4.0,3.0,6.0,1.0,14.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.000000,15.0,2.000000,4.0,5.0,8.0,0.0,23.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.000000,12.0,2.371956,3.0,5.0,3.0,0.0,11.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [166]:
print("Before: ", df_one_hot.shape)
print("After: ", df_imputed.shape)
print("Dropped: ", df_one_hot.shape[0] - df_imputed.shape[0])

Before:  (5630, 37)
After:  (5630, 37)
Dropped:  0


In [167]:
# Drop Target variable 
y_dropped = df_dropped["Churn"]
df_dropped = df_dropped.drop(columns=['Churn'],axis=1)
y_imputed = df_imputed["Churn"]
df_imputed = df_imputed.drop(columns=['Churn'],axis=1)
df.drop(columns=['Churn'],axis=1, inplace=True)

categorical_columns = ["PreferredLoginDevice", "CityTier", "PreferredPaymentMode", "Gender", "PreferedOrderCat", "MaritalStatus"] # for one hot encoding, exclude columns that are already binary
numerical_columns = [col for col in df.columns if col not in categorical_columns] # Churnhas been dropped above

In [168]:
df_dropped

,Tenure,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,...,Gender_Male,PreferedOrderCat_Fashion,PreferedOrderCat_Grocery,PreferedOrderCat_Laptop & Accessory,PreferedOrderCat_Mobile,PreferedOrderCat_Mobile Phone,PreferedOrderCat_Others,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
CustomerID,,,,,,,,,,,,,,,,,,,,,
50001,4.0,6.0,3.0,3,2,9,1,11.0,1.0,1.0,...,0,0,0,1,0,0,0,0,0,1
50004,0.0,15.0,2.0,4,5,8,0,23.0,0.0,1.0,...,1,0,0,1,0,0,0,0,0,1
50006,0.0,22.0,3.0,5,5,2,1,22.0,4.0,6.0,...,0,0,0,0,0,1,0,0,0,1
50012,11.0,6.0,3.0,4,3,10,1,13.0,0.0,1.0,...,1,1,0,0,0,0,0,0,0,1
50013,0.0,11.0,2.0,3,3,2,1,13.0,2.0,2.0,...,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55625,1.0,12.0,2.0,5,3,2,0,19.0,2.0,2.0,...,0,0,0,0,0,1,0,0,0,1
55626,10.0,30.0,3.0,2,1,6,0,18.0,1.0,2.0,...,1,0,0,1,0,0,0,0,1,0
55628,1.0,11.0,3.0,2,4,3,1,21.0,1.0,2.0,...,1,0,0,1,0,0,0,0,1,0


### Handle outliers

In [169]:
def drop_outliers(df_version, columns, threshold=1.5):
    df_no_outliers = df_version.copy()
    for col in columns:
        Q1 = df_no_outliers[col].quantile(0.25)
        Q3 = df_no_outliers[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        df_no_outliers = df_no_outliers[(df_no_outliers[col] > lower_bound) & (df_no_outliers[col] < upper_bound)]
    return df_no_outliers

# droped null values with outliers
df_dropped_with_outliers = df_dropped
# droped null values no outliers
df_dropped_no_outliers = drop_outliers(df_dropped, ['Tenure'])
# imputed null values with outliers
df_imputed_with_outliers = df_imputed
# imputed null values without outliers
df_imputed_no_outliers = drop_outliers(df_imputed, [x for x in numerical_columns if x != "Complain"])

all_df = [df_dropped_with_outliers, df_dropped_no_outliers, df_imputed_with_outliers, df_imputed_no_outliers]
for df in all_df:
    print(df.shape)

(3774, 36)
(3749, 36)
(5630, 36)
(3559, 36)


We cannot remove outliers from numerical variables as they are merely result of insufficient samples. This is because when we remove all outliers from all numerical columns, there will be few rows left. If we remove outliers, the accuracy of our model may be high, but will fail when we are met with outliers during inference, which we will as there are 2000+ rows with outliers in this dataset alone.

In [170]:
# Keep outliers
all_df = [df_dropped_with_outliers, df_imputed_with_outliers]

### Features Selection using Variational Inflation Factor

VIF is used to detect multicollinearity in numerical features. VIF value of more than 5 means high multicollinearity, which means the variables are redundant and can be removed.

In [171]:
def select_features_with_variance_threshold(data, threshold=5.0):

    numerical_data = data[[x for x in numerical_columns if x != "Complain"]]
    categorical_data = data.drop(columns=numerical_columns, errors='ignore')

    selector = VarianceThreshold(threshold)
    selector.fit(numerical_data)
    selected_numerical_features = numerical_data.columns[selector.get_support(indices=True)]

    selected_data = pd.concat([data[selected_numerical_features], categorical_data], axis=1)

    return selected_data

# Calculate VIF and select features < 5.0 VIF
df_feature_selected = []
for df in all_df:
    df_feature_selected.append(select_features_with_variance_threshold(df.drop(columns=["Complain"]))) # Exclude target column

# Print the result
for i in range(len(all_df)):
    excluded_columns = [col for col in all_df[i].columns if col not in df_feature_selected[i].columns]
    print(excluded_columns)

['HourSpendOnApp', 'NumberOfDeviceRegistered', 'SatisfactionScore', 'Complain', 'CouponUsed']
['HourSpendOnApp', 'NumberOfDeviceRegistered', 'SatisfactionScore', 'Complain', 'CouponUsed']


In [172]:
df_feature_selected[0].shape

(3774, 31)

These columns are multicollinear which means they are strongly correlated with each other, it will cause interpretation difficulties, reduce statistical power, and inefficient use of data.

We need to add the "Complain" column back as we dropped it before calculating VIF as it is a categorical variable. We also need to keep one variable from the list of multicollinear variables.

In [173]:
# Add Churn back to our data
for i in range(len(df_feature_selected)):
    df_feature_selected[i]["Complain"] = all_df[i]["Complain"]
    # We will add back satisfaction score as it is highly correlated to Churn
    df_feature_selected[i]['SatisfactionScore']=all_df[i]['SatisfactionScore']

print(df_feature_selected[0].shape)
print(df_feature_selected[1].shape)

(3774, 33)
(5630, 33)


### Standardizing

In [174]:
# Exclude removed features from numerical features
numerical_columns = [x for x in numerical_columns if x not in excluded_columns or x == "Complain"]
numerical_columns

['Tenure',
 'WarehouseToHome',
 'NumberOfAddress',
 'Complain',
 'OrderAmountHikeFromlastYear',
 'OrderCount',
 'DaySinceLastOrder',
 'CashbackAmount']

In [175]:
# Initialize StandardScaler
scaler = StandardScaler()
numerical_columns_to_scale = [x for x in numerical_columns if x != "Complain"]
# Standardize numerical columns
for df in df_feature_selected:
    df[numerical_columns_to_scale] = scaler.fit_transform(df[numerical_columns_to_scale]) # passed by reference

In [176]:
df_feature_selected[1]

,Tenure,WarehouseToHome,NumberOfAddress,OrderAmountHikeFromlastYear,OrderCount,DaySinceLastOrder,CashbackAmount,PreferredLoginDevice_Computer,PreferredLoginDevice_Mobile Phone,PreferredLoginDevice_Phone,...,PreferedOrderCat_Grocery,PreferedOrderCat_Laptop & Accessory,PreferedOrderCat_Mobile,PreferedOrderCat_Mobile Phone,PreferedOrderCat_Others,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Complain,SatisfactionScore
0,-0.706898,-1.153015,1.852616,-1.313321,-0.703971,0.098427,-0.351465,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0
1,-0.890138,-0.913258,1.078430,-0.198693,-0.703971,-1.283859,-1.144715,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0
2,-0.748441,1.724076,0.691336,-0.477350,-0.703971,-0.454487,-1.157316,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0
3,-1.180335,-0.074106,1.465523,2.030564,-0.703971,-0.454487,-0.877047,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0
4,-1.180335,-0.433742,-0.469944,-1.313321,-0.703971,-0.454487,-0.967895,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,0.003257,1.724076,0.691336,0.637278,-0.365836,-0.178030,-0.538854,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5626,0.358334,-0.313864,0.691336,0.079964,-0.365836,0.105811,0.969195,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0
5627,-1.061976,-0.553621,-0.469944,1.473250,-0.365836,-0.178030,0.186920,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,4.0
5628,1.541926,-0.793379,-0.082850,-0.198693,-0.365836,1.204255,0.034083,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0


# Model Training

### Train Test Split

In [178]:
# Train Test Splitting

# Dataset with null values dropped
X_train_dropped, X_test_dropped, y_train_dropped, y_test_dropped = train_test_split(df_feature_selected[0], y_dropped, test_size=0.2, random_state=42)
print("Set dropped: ", X_train_dropped.shape, X_test_dropped.shape)

# Dataset with null values imputed
X_train_imputed, X_test_imputed, y_train_imputed, y_test_imputed = train_test_split(df_feature_selected[1], y_imputed, test_size=0.2, random_state=42)
print("Set Imputed: ", X_train_imputed.shape, X_test_imputed.shape)

Set dropped:  (3019, 33) (755, 33)
Set Imputed:  (4504, 33) (1126, 33)


### Hyperparameter Grid Search

In [233]:
# Define parameters for each algorithm
svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
knn_params = {'n_neighbors': [3, 5, 7]}
rf_params = {'n_estimators': [100, 300], 'max_depth': [3, 5]}
xgb_params = {'learning_rate': [0.1, 0.01], 'max_depth': [3, 5]}
catboost_params = {'iterations': [100, 200], 'learning_rate': [0.03, 0.1]}
adaboost_params = {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1]}
logistic_params = {'C': [0.1, 1, 10]}
naive_bayes_params = {}  # No hyperparameters for GaussianNB

# Initialize models
svm = SVC()
knn = KNeighborsClassifier()
rf = RandomForestClassifier()
xgb = XGBClassifier()
catboost = CatBoostClassifier()
adaboost = AdaBoostClassifier()
logistic = LogisticRegression()
naive_bayes = GaussianNB()

# Define the list of classifiers and their corresponding parameter grids
classifiers = [
    (svm, svm_params),
    (knn, knn_params),
    (rf, rf_params),
    (xgb, xgb_params),
    (catboost, catboost_params),
    (adaboost, adaboost_params),
    (logistic, logistic_params),
    (naive_bayes, naive_bayes_params)
]

# Save best models
best_models = []

Training 1: 
- Dataset: Dropped null values
- Metrics: Accuracy

In [234]:
# Fit dataset 1: dropped
# Run GridSearchCV for each classifier
old_test_score = 0
for clf, params in classifiers:
    grid_search = GridSearchCV(clf, params, cv=5, scoring='accuracy')
    grid_search.fit(X_train_dropped, y_train_dropped)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_dropped)

    test_score = accuracy_score(y_test_dropped, y_pred)
    if test_score > old_test_score:
        best_classifier = grid_search.best_estimator_
    old_test_score = test_score
    print(f"Best parameters for {clf.__class__.__name__}: {grid_search.best_params_}")
    print(f"Best score for {clf.__class__.__name__}: {grid_search.best_score_}")
    print(f"Test score for {clf.__class__.__name__}: {test_score}")
    print("----")

best_models.append(best_classifier)

Best parameters for SVC: {'C': 10, 'kernel': 'rbf'}
Best score for SVC: 0.9284521103093801
Test score for SVC: 0.9456953642384106
----
Best parameters for KNeighborsClassifier: {'n_neighbors': 3}
Best score for KNeighborsClassifier: 0.9003003745071553
Test score for KNeighborsClassifier: 0.9178807947019868
----
Best parameters for RandomForestClassifier: {'max_depth': 5, 'n_estimators': 300}
Best score for RandomForestClassifier: 0.8675051892853614
Test score for RandomForestClassifier: 0.8940397350993378
----
Best parameters for XGBClassifier: {'learning_rate': 0.1, 'max_depth': 5}
Best score for XGBClassifier: 0.9357423698285615
Test score for XGBClassifier: 0.9589403973509933
----
0:	learn: 0.6653122	total: 2.87ms	remaining: 284ms
1:	learn: 0.6363250	total: 7.66ms	remaining: 375ms
2:	learn: 0.6085003	total: 10.8ms	remaining: 350ms
3:	learn: 0.5815500	total: 13.9ms	remaining: 332ms
4:	learn: 0.5591523	total: 17.5ms	remaining: 332ms
5:	learn: 0.5399927	total: 21ms	remaining: 328ms
6:	

d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

Best parameters for LogisticRegression: {'C': 1}
Best score for LogisticRegression: 0.8890371541849253
Test score for LogisticRegression: 0.9086092715231788
----
Best parameters for GaussianNB: {}
Best score for GaussianNB: 0.31600606240321566
Test score for GaussianNB: 0.2860927152317881
----


d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training 2: 
- Dataset: Dropped null values
- Metrics: F1

In [235]:
# Run GridSearchCV for each classifier
old_test_score = 0
for clf, params in classifiers:
    grid_search = GridSearchCV(clf, params, cv=5, scoring='f1')
    grid_search.fit(X_train_dropped, y_train_dropped)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_dropped)
    test_score = f1_score(y_test_dropped, y_pred)
    if test_score > old_test_score:
        best_classifier = grid_search.best_estimator_
    old_test_score = test_score
    print(f"Best parameters for {clf.__class__.__name__}: {grid_search.best_params_}")
    print(f"Best score for {clf.__class__.__name__}: {grid_search.best_score_}")
    print(f"Test score for {clf.__class__.__name__}: {test_score}")
    print("----")

best_models.append(best_classifier)

Best parameters for SVC: {'C': 10, 'kernel': 'rbf'}
Best score for SVC: 0.7779607910795625
Test score for SVC: 0.8038277511961723
----
Best parameters for KNeighborsClassifier: {'n_neighbors': 3}
Best score for KNeighborsClassifier: 0.6824677003764631
Test score for KNeighborsClassifier: 0.6960784313725489
----
Best parameters for RandomForestClassifier: {'max_depth': 5, 'n_estimators': 300}
Best score for RandomForestClassifier: 0.3972536967008758
Test score for RandomForestClassifier: 0.40579710144927533
----
Best parameters for XGBClassifier: {'learning_rate': 0.1, 'max_depth': 5}
Best score for XGBClassifier: 0.8014530786324926
Test score for XGBClassifier: 0.8442211055276382
----
0:	learn: 0.6653122	total: 1.97ms	remaining: 195ms
1:	learn: 0.6363250	total: 20.9ms	remaining: 1.02s
2:	learn: 0.6085003	total: 24.9ms	remaining: 806ms
3:	learn: 0.5815500	total: 27.7ms	remaining: 666ms
4:	learn: 0.5591523	total: 30.5ms	remaining: 579ms
5:	learn: 0.5399927	total: 33.4ms	remaining: 524ms


d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

Best parameters for LogisticRegression: {'C': 1}
Best score for LogisticRegression: 0.6228218837969729
Test score for LogisticRegression: 0.6229508196721311
----
Best parameters for GaussianNB: {}
Best score for GaussianNB: 0.3289638406117368
Test score for GaussianNB: 0.276510067114094
----


d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training 3: 
- Dataset: Imputed null values
- Metrics: Accuracy

In [236]:
# Run GridSearchCV for each classifier
old_test_score = 0
for clf, params in classifiers:
    grid_search = GridSearchCV(clf, params, cv=5, scoring='accuracy')
    grid_search.fit(X_train_imputed, y_train_imputed)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_imputed)

    test_score = accuracy_score(y_test_imputed, y_pred)
    if test_score > old_test_score:
        best_classifier = grid_search.best_estimator_
    old_test_score = test_score
    print(f"Best parameters for {clf.__class__.__name__}: {grid_search.best_params_}")
    print(f"Best score for {clf.__class__.__name__}: {grid_search.best_score_}")
    print(f"Test score for {clf.__class__.__name__}: {test_score}")
    print("----")

best_models.append(best_classifier)

Best parameters for SVC: {'C': 10, 'kernel': 'rbf'}
Best score for SVC: 0.9378352447897399
Test score for SVC: 0.9564831261101243
----
Best parameters for KNeighborsClassifier: {'n_neighbors': 3}
Best score for KNeighborsClassifier: 0.9247359723763718
Test score for KNeighborsClassifier: 0.9484902309058615
----
Best parameters for RandomForestClassifier: {'max_depth': 5, 'n_estimators': 100}
Best score for RandomForestClassifier: 0.8730038229128129
Test score for RandomForestClassifier: 0.8703374777975134
----
Best parameters for XGBClassifier: {'learning_rate': 0.1, 'max_depth': 5}
Best score for XGBClassifier: 0.9420557405352078
Test score for XGBClassifier: 0.9538188277087034
----
0:	learn: 0.6613528	total: 3.8ms	remaining: 376ms
1:	learn: 0.6306327	total: 6.87ms	remaining: 337ms
2:	learn: 0.5989073	total: 10.9ms	remaining: 353ms
3:	learn: 0.5728445	total: 17.5ms	remaining: 419ms
4:	learn: 0.5455030	total: 21.1ms	remaining: 401ms
5:	learn: 0.5245529	total: 24.6ms	remaining: 385ms
6:

d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

Best parameters for LogisticRegression: {'C': 10}
Best score for LogisticRegression: 0.8885453200147984
Test score for LogisticRegression: 0.9147424511545293
----
Best parameters for GaussianNB: {}
Best score for GaussianNB: 0.7195868787766679
Test score for GaussianNB: 0.7531083481349912
----


Training 4: 
- Dataset: Imputed null values
- Metrics: F1

In [237]:
# Run GridSearchCV for each classifier
old_test_score = 0
for clf, params in classifiers:
    grid_search = GridSearchCV(clf, params, cv=5, scoring='f1')
    grid_search.fit(X_train_imputed, y_train_imputed)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_imputed)

    test_score = accuracy_score(y_test_imputed, y_pred)
    if test_score > old_test_score:
        best_classifier = grid_search.best_estimator_
    old_test_score = test_score
    print(f"Best parameters for {clf.__class__.__name__}: {grid_search.best_params_}")
    print(f"Best score for {clf.__class__.__name__}: {grid_search.best_score_}")
    print(f"Test score for {clf.__class__.__name__}: {test_score}")
    print("----")

best_models.append(best_classifier)

Best parameters for SVC: {'C': 10, 'kernel': 'rbf'}
Best score for SVC: 0.8031118371700376
Test score for SVC: 0.9564831261101243
----
Best parameters for KNeighborsClassifier: {'n_neighbors': 3}
Best score for KNeighborsClassifier: 0.7616593679170818
Test score for KNeighborsClassifier: 0.9484902309058615
----
Best parameters for RandomForestClassifier: {'max_depth': 5, 'n_estimators': 100}
Best score for RandomForestClassifier: 0.41322987460135996
Test score for RandomForestClassifier: 0.8774422735346359
----
Best parameters for XGBClassifier: {'learning_rate': 0.1, 'max_depth': 5}
Best score for XGBClassifier: 0.814858081599759
Test score for XGBClassifier: 0.9538188277087034
----
0:	learn: 0.6613528	total: 4.21ms	remaining: 417ms
1:	learn: 0.6306327	total: 14.5ms	remaining: 710ms
2:	learn: 0.5989073	total: 18.4ms	remaining: 596ms
3:	learn: 0.5728445	total: 24.6ms	remaining: 590ms
4:	learn: 0.5455030	total: 30.8ms	remaining: 586ms
5:	learn: 0.5245529	total: 34.9ms	remaining: 547ms
6

d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

Best parameters for LogisticRegression: {'C': 10}
Best score for LogisticRegression: 0.6175505374553409
Test score for LogisticRegression: 0.9147424511545293
----
Best parameters for GaussianNB: {}
Best score for GaussianNB: 0.457496681038083
Test score for GaussianNB: 0.7531083481349912
----


d:\NoidAI\Projects\Ecommerce\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Evaluation

Comparing models trained on different datasets and metrics

In [238]:
best_models

[LogisticRegression(C=1),
 LogisticRegression(C=1),
 LogisticRegression(C=10),
 LogisticRegression(C=10)]

In [249]:
predictions = best_models[3].predict(X_test_dropped)
f1_score(predictions, y_test_dropped)

0.5730337078651686

### Training with Grid Search Cross Validation

### 

### Creating pipeline